In [4]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import os, json, requests
import json
import ssl
import plotly.express as px
import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys

import pymannkendall as mk
import pyhomogeneity as hg
# import climdex.precipitation as pdex
# import xarray as xr
# import datetime
import plotly.express as px
import datetime


In [5]:

longitude = -48.3382
latitude = -21.8045

endtime = str(int(datetime.date.today().strftime("%Y"))-1)+'1231'
base_url = (r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M_MAX,PRECTOT,T2M_MIN&community=RE&longitude={longitude}&latitude={latitude}&start=19810101&end="+endtime+"&format=JSON")
api_request_url = base_url.format(longitude=longitude, latitude = latitude)
response = requests.get(url=api_request_url, verify=True, timeout=1000)
content = json.loads(response.content.decode('utf-8'))
df = pd.DataFrame.from_dict(content['properties']['parameter'])

In [4]:
df

,T2M_MAX,T2M_MIN,PRECTOTCORR
19810101,29.34,15.95,0.41
19810102,29.83,16.98,1.21
19810103,28.21,19.22,4.21
19810104,28.58,19.33,4.57
19810105,26.83,19.91,2.99
...,...,...,...
20221227,28.48,19.74,16.39
20221228,28.29,20.32,40.16
20221229,26.27,19.63,11.78
20221230,30.00,18.63,4.82


In [6]:
# Set the 'Date' column as the index
df = df.reset_index().rename(columns={'index': 'Date'})
df['Date'] =  pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df

,T2M_MAX,T2M_MIN,PRECTOTCORR
Date,,,
1981-01-01,29.34,15.96,0.41
1981-01-02,29.83,16.97,1.21
1981-01-03,28.21,19.22,4.22
1981-01-04,28.58,19.33,4.57
1981-01-05,26.83,19.91,2.99
...,...,...,...
2024-12-27,29.46,20.97,7.05
2024-12-28,29.02,21.36,4.78
2024-12-29,30.90,19.91,5.10


In [7]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import os, json, requests
import json
import ssl
import plotly.express as px
import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots

longitude = -48.3382
latitude = -21.8045


base_url = (r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M_MAX,PRECTOT,T2M_MIN&community=RE&longitude={longitude}&latitude={latitude}&start=19810101&end=20221231&format=JSON")
api_request_url = base_url.format(longitude=longitude, latitude=latitude)
response = requests.get(url=api_request_url, verify=True, timeout=1000)
content = json.loads(response.content.decode('utf-8'))
df = pd.DataFrame.from_dict(content['properties']['parameter'])
df = df.reset_index().rename(columns={'index': 'Date'})
df['Date'] =  pd.to_datetime(df['Date'])
df_aux = df.copy()       
df.set_index('Date', inplace=True)
ds = df[['PRECTOTCORR', 'T2M_MAX', 'T2M_MIN']].copy().to_xarray()

In [9]:
import numpy as np
np.bool = np.bool_
precip_indices.cwd(ds, period='1M', varname='PRECTOTCORR').to_dataframe()

NameError: name 'precip_indices' is not defined

In [53]:
df_aux['Accumulated_Precipitation'] = df_aux['PRECTOTCORR'].rolling(window=90).sum()
df_aux.dropna(inplace=True)
params = gamma.fit(df_aux['Accumulated_Precipitation'], floc=0)
df_aux['Cumulative_Probability'] = gamma.cdf(df_aux['Accumulated_Precipitation'], *params)
# Transform the cumulative probabilities to the standard normal distribution to get the SPI values
df_aux['SPI'] = norm.ppf(df_aux['Cumulative_Probability'])
# Display the first few rows with the accumulated precipitation and SPI values
df_aux[['SPI']].describe()

,SPI
count,15073.000000
mean,0.003318
std,1.000882
min,-3.810166
25%,-0.738852
50%,0.050258
75%,0.824429
max,2.117552


In [49]:
df_aux.set_index('Date', inplace=True)
df_aux

,T2M_MAX,T2M_MIN,PRECTOTCORR,Accumulated_Precipitation,Cumulative_Probability,SPI
Date,,,,,,
1981-03-31,27.29,16.73,0.02,496.17,0.802698,0.851297
1981-04-01,27.01,15.44,0.03,495.79,0.802370,0.850117
1981-04-02,27.05,15.96,0.02,494.60,0.801340,0.846416
1981-04-03,27.45,15.72,0.75,491.14,0.798317,0.835624
1981-04-04,27.65,15.01,1.11,487.68,0.795253,0.824783
...,...,...,...,...,...,...
2022-12-27,28.48,19.74,16.39,348.62,0.632937,0.339642
2022-12-28,28.29,20.32,40.16,387.69,0.686921,0.487142
2022-12-29,26.27,19.63,11.78,397.97,0.699968,0.524308


,SPI
Date,
1981-03-31,0.851297
1981-04-01,0.850117
1981-04-02,0.846416
1981-04-03,0.835624
1981-04-04,0.824783
...,...
2022-12-27,0.339642
2022-12-28,0.487142
2022-12-29,0.524308


In [37]:
precip_indices = pdex.indices(time_dim='Date')

rx1day = precip_indices.monthly_rx1day(ds, varname='PREC')
rx5day = precip_indices.monthly_rx5day(ds, varname='PREC')
r10mm = precip_indices.annual_r10mm(ds, varname='PREC')
r20mm = precip_indices.annual_r20mm(ds, varname='PREC')
prcptot_annual = precip_indices.prcptot(ds, period='1Y', varname='PREC')
sdii_value = precip_indices.sdii(ds, period='1M', varname='PREC')
#cdd_value = precip_indices.cdd(ds, period='1M', varname='PREC')
#cwd_value = precip_indices.cwd(ds, period='1M', varname='PREC')

KeyError: 'PREC'

In [39]:
ds

<xarray.Dataset>
Dimensions:  (Date: 15340)
Coordinates:
  * Date     (Date) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Data variables:
    PREC     (Date) float64 0.41 1.21 4.21 4.57 2.99 ... 40.16 11.78 4.82 4.87

In [9]:
rx5day = rx5day.to_dataframe()
rx5day.plot


In [10]:
print("Monthly Maximum 1-day Precipitation:\n", rx1day)
print("\nMonthly Maximum 5-day Precipitation:\n", rx5day)
print("\nAnnual Count of Days when Precipitation Exceeds 10mm:\n", r10mm)
print("\nAnnual Count of Days when Precipitation Exceeds 20mm:\n", r20mm)
print("\nTotal Annual Precipitation:\n", prcptot_annual)
print("\nSimple Precipitation Intensity Index:\n", sdii_value)
# print("\nNumber of Consecutive Dry Days in a Month:\n", cdd_value)
# print("\nNumber of Consecutive Wet Days in a Month:\n", cwd_value)

Monthly Maximum 1-day Precipitation:
 <xarray.DataArray 'PREC' (Date: 504)>
array([2.704e+01, 1.143e+01, 1.742e+01, 1.760e+01, 4.660e+00, 2.818e+01,
       4.900e-01, 2.540e+00, 1.173e+01, 4.452e+01, 2.600e+01, 2.775e+01,
       4.554e+01, 3.840e+01, 3.646e+01, 3.112e+01, 1.958e+01, 1.791e+01,
       8.450e+00, 4.970e+00, 1.381e+01, 3.217e+01, 2.488e+01, 2.572e+01,
       3.468e+01, 6.184e+01, 3.700e+01, 4.101e+01, 5.376e+01, 3.181e+01,
       1.672e+01, 7.700e-01, 3.542e+01, 3.085e+01, 2.022e+01, 3.009e+01,
       3.658e+01, 1.801e+01, 1.580e+01, 9.650e+00, 1.144e+01, 4.000e-02,
       1.560e+00, 3.822e+01, 2.958e+01, 1.794e+01, 2.195e+01, 3.732e+01,
       3.787e+01, 2.427e+01, 3.482e+01, 3.017e+01, 2.328e+01, 1.121e+01,
       3.950e+00, 7.700e+00, 6.010e+00, 1.463e+01, 5.559e+01, 3.366e+01,
       4.927e+01, 2.907e+01, 2.814e+01, 1.907e+01, 3.038e+01, 9.000e-02,
       1.973e+01, 4.593e+01, 1.490e+01, 2.190e+01, 2.617e+01, 4.341e+01,
       5.124e+01, 2.639e+01, 3.141e+01, 1.622e+0

In [34]:
precip_indices.cwd(ds, period='1M', varname='PRECTOTCORR').to_dataframe()

KeyError: 'PRECTOTCORR'

,PREC,Accumulated_Precipitation,Cumulative_Probability,SPI
Date,,,,
1981-03-31,0.02,496.17,0.802698,0.851297
1981-04-01,0.03,495.79,0.802370,0.850117
1981-04-02,0.02,494.60,0.801340,0.846416
1981-04-03,0.75,491.14,0.798317,0.835624
1981-04-04,1.11,487.68,0.795253,0.824783


In [30]:
df_aux

,Date,T2M_MAX,T2M_MIN,PREC,Accumulated_Precipitation
89,1981-03-31,27.29,16.73,0.02,496.17
90,1981-04-01,27.01,15.44,0.03,495.79
91,1981-04-02,27.05,15.96,0.02,494.60
92,1981-04-03,27.45,15.72,0.75,491.14
93,1981-04-04,27.65,15.01,1.11,487.68
...,...,...,...,...,...
15335,2022-12-27,28.48,19.74,16.39,348.62
15336,2022-12-28,28.29,20.32,40.16,387.69
15337,2022-12-29,26.27,19.63,11.78,397.97
15338,2022-12-30,30.00,18.63,4.82,401.35


In [19]:
df = df.dropna()
from scipy.stats import gamma, norm



In [20]:
# Calculate the cumulative probabilities for the gamma distribution
df['Cumulative_Probability'] = gamma.cdf(df['Accumulated_Precipitation'], *params)

# Transform the cumulative probabilities to the standard normal distribution to get the SPI values
df['SPI'] = norm.ppf(df['Cumulative_Probability'])

# Display the first few rows with the accumulated precipitation and SPI values
df.head()

C:\Users\pc\AppData\Local\Temp\ipykernel_8304\432768635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cumulative_Probability'] = gamma.cdf(df['Accumulated_Precipitation'], *params)
C:\Users\pc\AppData\Local\Temp\ipykernel_8304\432768635.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SPI'] = norm.ppf(df['Cumulative_Probability'])


,T2M_MIN,T2M_MAX,PREC,Accumulated_Precipitation,Cumulative_Probability,SPI
Date,,,,,,
1981-03-31,16.73,27.29,0.02,496.17,0.802698,0.851297
1981-04-01,15.44,27.01,0.03,495.79,0.802370,0.850117
1981-04-02,15.96,27.05,0.02,494.60,0.801340,0.846416
1981-04-03,15.72,27.45,0.75,491.14,0.798317,0.835624
1981-04-04,15.01,27.65,1.11,487.68,0.795253,0.824783


In [22]:
df= df.reset_index()
df

,Date,T2M_MIN,T2M_MAX,PREC,Accumulated_Precipitation,Cumulative_Probability,SPI
0,1981-03-31,16.73,27.29,0.02,496.17,0.802698,0.851297
1,1981-04-01,15.44,27.01,0.03,495.79,0.802370,0.850117
2,1981-04-02,15.96,27.05,0.02,494.60,0.801340,0.846416
3,1981-04-03,15.72,27.45,0.75,491.14,0.798317,0.835624
4,1981-04-04,15.01,27.65,1.11,487.68,0.795253,0.824783
...,...,...,...,...,...,...,...
15246,2022-12-27,19.74,28.48,16.39,348.62,0.632937,0.339642
15247,2022-12-28,20.32,28.29,40.16,387.69,0.686921,0.487142
15248,2022-12-29,19.63,26.27,11.78,397.97,0.699968,0.524308
15249,2022-12-30,18.63,30.00,4.82,401.35,0.704155,0.536390


In [23]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['SPI'], mode='lines+markers', name='Value over Time'))

# Customizing the layout (optional)
fig.update_layout(title='Line plot from DataFrame', xaxis_title='Date', yaxis_title='Value')

# Show the plot
fig.show()

In [11]:
df_aux = df.groupby([df.Date.dt.year, df.Date.dt.month]).sum(numeric_only=True)[['PRECTOTCORR']]
#df_aux = df.groupby(df['Date'].dt.year)[['PRECTOTCORR']].sum()
#df = df.groupby([(df.Date.dt.year), (df.Date.dt.month)]).mean()[['T2M_MIN','T2M_MAX']]

AttributeError: 'DataFrame' object has no attribute 'Date'

In [ ]:
df.groupby([(df.Date.dt.year), (df.Date.dt.month)]).mean(numeric_only=True)[['T2M_MIN','T2M_MAX']]

T2M_MIN    T2M_MAX
Date Date                      
1981 1     19.215161  27.940645
     2     18.717500  28.535000
     3     18.068387  28.585484
     4     14.854667  26.854667
     5     13.886129  26.553548
...              ...        ...
2022 8     13.588065  27.690968
     9     14.938333  30.290667
     10    18.221935  32.231935
     11    16.489667  30.649000
     12    18.761935  29.246129

[504 rows x 2 columns]

In [ ]:




df['PRECTOTCORR'] = df_aux['PRECTOTCORR']
df.reset_index(level=1, inplace=True)
df = df.groupby(df.Date).mean()
df.reset_index(inplace=True)
df.rename(columns = {'Date':'Month'}, inplace = True)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=df['Month'], y=df['PRECTOTCORR'], name='PRECTOTCORR'),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x=df['Month'], y=df['T2M_MAX'], name='T2M_MAX'),
    secondary_y=True,
)

# Add figure title
# fig.update_layout(
#     title_text="Double Y Axis Example"
# )

# Set x-axis title
fig.update_xaxes(title_text="month")

fig.update_layout(
xaxis = dict(
    tickmode = 'linear',
)
)

# Set y-axes titles
fig.update_yaxes(title_text="T2M_MAX (ºC)", secondary_y=True)
fig.update_yaxes(title_text="PRECTOTCORR (mm)", secondary_y=False)

myFile = io.StringIO()
config = {'displaylogo': False,'modeBarButtonsToRemove': ['toImage','toggleHover']}
fig.write_html(myFile, config = config)
html = myFile.getvalue()  
self.webView_4.setHtml(html)
print('ok plot2')


In [ ]:
df = df[df['PRECTOTCORR'] > 0]

In [ ]:
df[df.Date > '2022-01-15']

In [ ]:
ax = df[df.Date > '2022-01-15']
prec = ax[ax['PRECTOTCORR'] > 0][['PRECTOTCORR']].sum()[0]

In [ ]:
prec

893.1700000000001

In [ ]:
type(prec)

numpy.float64

In [ ]:
import io
df = df.reset_index().rename(columns={'index': 'Date'})
df['Date'] =  pd.to_datetime(df['Date'])
df_aux = df.groupby(df['Date'].dt.year).sum()[['PRECTOTCORR']]
df = df.groupby(df['Date'].dt.year).mean()[['T2M_MIN','T2M_MAX']]
df['PRECTOTCORR'] = df_aux['PRECTOTCORR']
df.reset_index(inplace=True)



In [ ]:
atributo = 'T2M_MAX'

In [ ]:
df_aux = df[['Date', atributo]].copy()
df_aux['Date'] = (df_aux['Date'].astype(str)+'-01-01').apply(pd.to_datetime)
df_aux.index = df_aux['Date']
df_aux = df_aux[[atributo]].astype(float)
result_mk = mk.original_test(df_aux)
result_pettitt = hg.pettitt_test(df_aux)

In [ ]:
title1 = 'Mann Kendall Test: trend='+result_mk.trend+', alpha=0.05'+', p-value='+str(result_mk.p)
title2 = None
if result_pettitt.h:
    title2 = 'data is nonhomogeneous, probable change point location='+result_pettitt.cp[:4]+', alpha=0.05'+', p-value='+str(result_pettitt.p)
else:
    title2 = 'data is homogeneous, alpha=0.05'+', p-value='+str(result_pettitt.p)

title = title1+'<br>'+title2
title


'Mann Kendall Test: trend=increasing, alpha=0.05, p-value=2.795538663868946e-05<br>data is nonhomogeneous, probable change point location=2004, alpha=0.05, p-value=0.001'

In [ ]:
df_aux = df[['Date', atributo]].copy()
df_aux['Date'] = (df_aux['Date'].astype(str)+'-01-01').apply(pd.to_datetime)
df_aux.index = df_aux['Date']
df_aux = df_aux[[atributo]].astype(float)
result_mk = mk.original_test(df_aux)
result_pettitt = hg.pettitt_test(df_aux)

title1 = 'Mann Kendall Test: trend='+result_mk.trend+', alpha=0.05'+', p-value='+str(result_mk.p)
title2 = None
if result_pettitt.h:
    title2 = 'Pettitt Test: data is nonhomogeneous, probable change point location='+result_pettitt.cp[:4]+', alpha=0.05'+', p-value='+str(result_pettitt.p)
else:
    title2 = 'Pettitt Test: data is homogeneous, alpha=0.05'+', p-value='+str(result_pettitt.p)

title = title1+'<br>'+title2

myFile = io.StringIO()

mode = ["toImage",    "sendDataToCloud",    "zoom2d",    "pan2d",    "select2d",    "lasso2d",    "zoomIn2d",    "zoomOut2d",
    "autoScale2d",
    "resetScale2d",
    "hoverClosestCartesian",
    "hoverCompareCartesian",
    "zoom3d",
]

fig = px.line(df, x="Date", y=[atributo], title='<b>'+atributo+'</b>'+'<br>'+title)
fig.update_layout(showlegend=False)
fig.write_html(myFile, config = {'displaylogo': False})
html = myFile.getvalue()  

fig

In [ ]:
df = df.reset_index().rename(columns={'index': 'Date'})
df['Date'] =  pd.to_datetime(df['Date'])
df_aux = df.groupby([(df.Date.dt.year), (df.Date.dt.month)]).sum()[['PRECTOTCORR']]
df = df.groupby([(df.Date.dt.year), (df.Date.dt.month)]).mean()[['T2M_MIN','T2M_MAX']]
df['PRECTOTCORR'] = df_aux['PRECTOTCORR']
df.reset_index(level=1, inplace=True)
df = df.groupby(df.Date).mean()
df.reset_index(inplace=True)
df.rename(columns = {'Date':'Month'}, inplace = True)

In [ ]:
import numpy, sympy
import pymannkendall as mk
# import pyhomogeneity as hg


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=df['Month'], y=df['PRECTOTCORR'], name='PRECTOTCORR'),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x=df['Month'], y=df['T2M_MAX'], name='T2M_MAX'),
    secondary_y=True,
)

# Add figure title
# fig.update_layout(
#     title_text="Double Y Axis Example"
# )

# Set x-axis title
fig.update_xaxes(title_text="month")

fig.update_layout(
   xaxis = dict(
      tickmode = 'linear',
   )
)

# Set y-axes titles
fig.update_yaxes(title_text="T2M_MAX (ºC)", secondary_y=True)
fig.update_yaxes(title_text="PRECTOTCORR (mm)", secondary_y=False)

fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Create a sample plot
df = pd.DataFrame({
    'x': [1, 2, 3],
    'y': [4, 5, 6]
})

fig = px.scatter(df, x='x', y='y')

# Add a menu with translated labels
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label='French',
                    method='update',
                    args=[
                        {'title': 'Titre'},
                        {'xaxis_title': 'Abscisses'},
                        {'yaxis_title': 'Ordonnées'},
                    ]
                ),
                dict(
                    label='German',
                    method='update',
                    args=[
                        {'title': 'Titel'},
                        {'xaxis_title': 'Abszissen'},
                        {'yaxis_title': 'Ordinaten'},
                    ]
                )
            ]
        )
    ]
)

fig.show()


In [ ]:
fig.show()

In [ ]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import os, json, requests
import json
import ssl
import plotly.express as px
import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots


longitude = -48.3382
latitude = -21.8045

base_url = (r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M_MAX,PRECTOT,T2M_MIN&community=RE&longitude={longitude}&latitude={latitude}&start=19810101&end=20211231&format=JSON")
api_request_url = base_url.format(longitude=longitude, latitude=latitude)
response = requests.get(url=api_request_url, verify=True, timeout=1000)
content = json.loads(response.content.decode('utf-8'))
df = pd.DataFrame.from_dict(content['properties']['parameter'])

In [ ]:
# df = self.df_nasa
df = df.reset_index().rename(columns={'index': 'Date'})
df['Date'] =  pd.to_datetime(df['Date'])
df['Estiagem'] = df['PRECTOTCORR'].map(lambda x: x <= 10.)
df['Estiagem'] = df['Estiagem'].astype(int)
resp = df.groupby(df['Date'].dt.year).sum()[['Estiagem']]

fig = px.bar(resp)
fig.update_layout(yaxis_range=[resp.min()[0]-1,resp.max()[0]+1])

In [ ]:
resp.min()[0]

282

In [ ]:
fig = px.bar(resp)
fig.update_layout(yaxis_range=[resp.min()[0]-1,resp.max()[0]+1])
fig.show()

In [ ]:
df['PRECTOTCORR'].map(lambda x: x <= 10.)


0         True
1         True
2         True
3         True
4         True
         ...  
14970     True
14971    False
14972     True
14973     True
14974     True
Name: PRECTOTCORR, Length: 14975, dtype: bool

,Estiagem
Date,
1981,315
1982,303
1983,282
1984,335
1985,322
1986,312
1987,314
1988,319
1989,317


In [ ]:
resp

,i10
Date,
1981,0
1982,0
1983,0
1984,0
1985,0
1986,0
1987,0
1988,0
1989,0


In [ ]:
df['i20'] = df['T2M_MAX'].map(lambda x: x <= 20.)
df['i20'] = df['i20'].astype(int)
resp = df.groupby(df['Date'].dt.year).sum()[['i20']]
resp

,i20
Date,
1981,12
1982,4
1983,12
1984,1
1985,8
1986,3
1987,14
1988,16
1989,11


In [ ]:

import urllib.request

def connect(host='http://google.com'):
    try:
        urllib.request.urlopen(host) #Python 3.x
        return True
    except:
        return False

# test
print( "connected" if connect() else "no internet!" )

connected


In [ ]:
import time
import sys
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
from config import LOGIN, URL_TEMPLATE
from datetime import datetime
from station import Station


class InputError(Exception):
    """Exception raised for errors in the input."""
    pass


def init_webdriver_config(impl_delay=30):
    """Function to set the configuration of the Selenium web Driver."""

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

    chrome_driver = webdriver.Chrome(chrome_options=chrome_options)
    chrome_driver.implicitly_wait(impl_delay)
    return chrome_driver


def get_page(_driver, _url):
    """Helper function that navigates and returns an BeautifulSoup page."""
    _driver.get(_url)
    time.sleep(5)
    return BeautifulSoup(_driver.page_source, 'html.parser')


def inmet_login(_driver):
    """This function navigates to the login page and do the login."""

    get_page(_driver, LOGIN['url'])

    _driver.find_element_by_name("mCod").send_keys(LOGIN['username'])
    _driver.find_element_by_name("mSenha").send_keys(LOGIN['password'])
    _driver.find_element_by_name("btnProcesso").click()


def get_url_pattern():
    """This function returns the URL pattern accordingly to the Template passed as system parameter."""

    if len(sys.argv) > 1:
        if sys.argv[1].upper() in URL_TEMPLATE.keys():
            print('TEMPLATE: ', sys.argv[1].upper())
            return URL_TEMPLATE[sys.argv[1].upper()]
        else:
            raise InputError('The template {0} don´t exist.'.format(sys.argv[1].upper()))

    print('TEMPLATE Default: MONTH')
    return URL_TEMPLATE['MONTH']


def load_station_numbers(_path):
    """It returns the list of stations."""

    with open(_path) as _file:
        _station_numbers = _file.readlines()
        _station_numbers = list(map(lambda x: x.replace("\n", ""), _station_numbers))

    return _station_numbers


if __name__ == '__main__':

    print(">> WebCrawler Started <<")

    count_success = 0
    count_error = 0
    station_list = []

    driver = init_webdriver_config(30)

    try:
        inmet_login(driver)

        url_pattern = get_url_pattern()

        station_numbers = load_station_numbers("./data/station_numbers.txt")

        start_date = "01/01/2018"
        end_date = datetime.now().strftime("%d/%m/%Y")

        for omm_code in station_numbers:
            url = url_pattern.format(omm_code=omm_code, start_date=start_date, end_date=end_date)
            soup = get_page(driver, url)

            soup_pre = soup.select('pre')

            if len(soup_pre) == 0:  # In case of an exception go to the next station
                continue

            if 'Não existem dados disponiveis' in soup_pre[0].text:
                count_error += 1
            else:
                content = soup_pre[0].text.split('--------------------')
                station = Station.parser(content[2])
                station.set_observation(Station.observation_parser(content[4]))
                station_list.append(station)
                count_success += 1

    except InputError as e:
        print(e)
    except Exception as e:
        print(e)
    finally:
        driver.quit()
        print(">> WebCrawler Finished <<")
        print("SUCCESS: {0}\nERROR: {1}\nTOTAL: {2}".format(count_success, count_error, count_success + count_error))

    if len(station_list) == 0:
        print('No data collected. Exiting...')
        exit()

    print(">> Data Processing Started <<")

    data_header = list(filter(None, station_list[0].weather_observation_header.split(';')))
    header = Station.get_station_header() + data_header

    file_path = 'data/output_data.csv'
    with open(file_path, 'w', newline='\n', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';', quotechar="'", quoting=csv.QUOTE_MINIMAL)

        writer.writerow(header)

        for station in station_list:
            for ob in station.weather_observation:
                row = station.get_station_information() + ob.split(';')
                writer.writerow(row)

        print('Saving the file {0} [...]'.format(file_path))

    print(">> Data Processing Finished <<")

ModuleNotFoundError: No module named 'selenium'